## Defasagem Poder de Compra da remuneração do Servidores

In [1]:
# !pip3 install pandas-ods-reader
# %pip install ezodf

In [2]:
list1 =  ['ESPECIAL', 'lll', 3703.72, 872.0, None, 752.0, 827.0, 902.0, 1462.0, 2925.0, None, 4575.72, 5327.72, 5402.72, 5477.72, 6037.72, 7500.72, None, 545.0, None, 4248.72, 5000.72, 5075.72, 5150.72, 5710.72, 7173.72, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
print(len(list1))

columns = ['PADRAO', 'NIVEL', 'A', 'B', 'VZ1' , 'C','D' ,'E', 'F','G=(A+B)', 'VZ2', 'H=(A+B+D)', 'I=(A+B+E)', 'J=(A+B+F)', 'VZ5', 'K=(A+C)', 'L=(A+C+D)', 'M=(A+C+E)', 'VZ3', 'N=(A+C+F)', 'VZ4', 'O', 'P=(A+O)', 'Q=(A+D+O)', 'R=(A+E+O)', 'S=(A+F+O)']
print(len(columns))

for n,i in enumerate(zip(columns,list1)):
    print(f'{n+1:2}: {i}')

53
26
 1: ('PADRAO', 'ESPECIAL')
 2: ('NIVEL', 'lll')
 3: ('A', 3703.72)
 4: ('B', 872.0)
 5: ('VZ1', None)
 6: ('C', 752.0)
 7: ('D', 827.0)
 8: ('E', 902.0)
 9: ('F', 1462.0)
10: ('G=(A+B)', 2925.0)
11: ('VZ2', None)
12: ('H=(A+B+D)', 4575.72)
13: ('I=(A+B+E)', 5327.72)
14: ('J=(A+B+F)', 5402.72)
15: ('VZ5', 5477.72)
16: ('K=(A+C)', 6037.72)
17: ('L=(A+C+D)', 7500.72)
18: ('M=(A+C+E)', None)
19: ('VZ3', 545.0)
20: ('N=(A+C+F)', None)
21: ('VZ4', 4248.72)
22: ('O', 5000.72)
23: ('P=(A+O)', 5075.72)
24: ('Q=(A+D+O)', 5150.72)
25: ('R=(A+E+O)', 5710.72)
26: ('S=(A+F+O)', 7173.72)


In [3]:
import os, re, csv, json, ezodf # type: ignore
import zipfile, requests, tempfile
import altair as alt
import pandas as pd
from io import BytesIO, TextIOWrapper
from bs4 import BeautifulSoup

def download_ods_files(url, save_folder):
    """Baixa todos os arquivos .ods de uma página da web e salva em uma pasta local.

    Args:
        url: A URL da página da web.
        save_folder: O caminho da pasta local onde os arquivos serão salvos.
    """
    # Cria a pasta de destino, se ela não existir
    # os.makedirs(save_folder, exist_ok=True)

    # Verifica se a pasta de destino existe
    if not os.path.exists(save_folder):
        # Cria a pasta de destino
        os.makedirs(save_folder)
        print(f"Pasta '{save_folder}' criada com sucesso!")
    else:
        print(f"Pasta '{save_folder}' já existe.")

    # Faz a requisição à página
    response = requests.get(url)
    response.raise_for_status()  # Verifica se a requisição foi bem-sucedida 
    status = response.status_code
    if response.status_code == 200:
        print('Página acessada com sucesso...')

    # Analisa o HTML da página
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontra todos os links que terminam em .ods
    # https://www.gov.br/servidor/pt-br/observatorio-de-pessoal-govbr/arquivos/tab-remunjul23-vol83ods-2.zip/@@download/file
    ods_links = [link['href'] for link in soup.find_all('a', href=True) if (link['href'].endswith('.zip') or link['href'].endswith('.zip/@@download/file'))]
    print(f'{len(ods_links)} links encontrados na página...')

    # Baixa e salva cada arquivo .ods
    for link in ods_links:
        linkname = os.path.basename(link)
        if linkname.endswith('.zip'):
            filename = linkname  # Obtém o nome do arquivo
        else:
            filename = linkname+'.zip'
        save_path = os.path.join(save_folder, filename)  # Caminho completo para salvar

        # Faz a requisição para baixar o arquivo
        response = requests.get(link)
        response.raise_for_status()

        # Salva o arquivo na pasta local
        with open(save_path, 'wb') as file:
            file.write(response.content)

        print(f"Arquivo {filename} baixado com sucesso!")

def extract_data_from_zip(folder_path, search_string, columns_to_extract):
    all_data = []
    df = pd.DataFrame()
    for filename in os.listdir(folder_path):
        if filename.endswith('.zip'):
            zip_file_path = os.path.join(folder_path, filename)
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                main_folder = zip_ref.namelist()[0]

                for item in zip_ref.namelist():
                    if item.startswith(main_folder) and search_string in item and item.endswith('/'):
                        subfolder_name = item
                        break
                else:
                    print(f"Subpasta com '{search_string}' não encontrada em {filename}.")
                    continue

                for name in zip_ref.namelist():
                    if name.startswith(subfolder_name) and name.endswith('.ods'):
                        with zip_ref.open(name) as file:
                            try:
                                ods_data = BytesIO(file.read())

                                with tempfile.NamedTemporaryFile(delete=False, suffix='.ods') as temp:
                                    temp.write(ods_data.getvalue())
                                    temp_path = temp.name

                                doc = ezodf.opendoc(temp_path)
                                sheet = doc.sheets[0]
                                # Obtém o valor da célula 0,0 que contém o nome do órgão em cada planilha
                                orgao = sheet[0, 0].value.split('.')[1].strip()
                                data = []
                                for i, row in enumerate(sheet.rows()):
                                    niv=None
                                    pos=None
                                    car=None
                                    classe_ant=None
                                    cols = [cell.value for cell in row][:26]
                                    if cols[0] in ['ESPECIAL', 'C', 'B', 'A']:
                                        classe_ant = cols[0]
                                        print(f'Classe anter: {classe_ant}')
                                    padrao = cols[1]
                                    for val in cols:
                                        if 'Cargo:' in str(val) and val[0] != '*':
                                            try:
                                                car = str(val).split(':')[1].strip()
                                                # print(f'Cargo: {car}')
                                            except:
                                                pass
                                        if 'Nível' in str(val):
                                            if '-' in str(val):
                                                niv = val.split('-')[0].strip()
                                            else:
                                                niv = str(val)
                                        if 'Posição:' in str(val):
                                            pos = val.split(':')[1].strip()
                                    
                                    # Coletar linhas de rótulos de cargo, nível e data da posição
                                    if (niv and pos) is not None:
                                        print(f'\n{orgao} | {pos} | {niv}')
                                    if car is not None:
                                        print(f'Cargo: {car}')
                                    
                                    # Coletar linhas de dados de valores
                                    if padrao in ['ÚNICO', 'VI', 'V', 'IV', 'lll', 'll', 'l']:
                                        print(f'Classe anter: {cols[0]}')
                                        print(cols)
                                        if cols[0] is None:
                                            cols[0] = classe_ant
                                            print(f'Classe atual: {cols[0]}')
                                        print(cols)
                                        data.append(cols[:26])

                                # Mapeia os nomes das colunas para os nomes corretos
                                column_mapping = {
                                    'PADRAO': 'PADRAO',
                                    'NIVEL': 'NIVEL',
                                    'A': 'VB',
                                    'B': None,
                                    'VZ1': None,
                                    'C': None,
                                    'D': None,
                                    'E': None,
                                    'F': None,
                                    'G=(A+B)': None,
                                    'VZ2': None,
                                    'H=(A+B+D)': None,
                                    'I=(A+B+E)': None,
                                    'J=(A+B+F)': None,
                                    'VZ5': None,
                                    'K=(A+C)': 'Ativo 100p Sem RT',
                                    'L=(A+C+D)': 'Ativo 100p Aperf./Espec.',
                                    'M=(A+C+E)': 'Ativo 100p Mestre',
                                    'N=(A+C+F)': 'Ativo 100p Doutor',
                                    'O': None,
                                    'P=(A+O)': 'Aposentado Sem RT',
                                    'Q=(A+D+O)': 'Aposentado Aperf./Espec.',
                                    'R=(A+E+O)': 'Aposentado Mestre', 
                                    'S=(A+F+O)': 'Aposentado Doutor',
                                }

                                columns = ['PADRAO', 'NIVEL', 'A', 'B', 'VZ1' , 'C','D' ,'E', 'F','G=(A+B)', 'VZ2', 'H=(A+B+D)', 'I=(A+B+E)', 'J=(A+B+F)', 'VZ5', 'K=(A+C)', 'L=(A+C+D)', 'M=(A+C+E)', 'VZ3', 'N=(A+C+F)', 'VZ4', 'O', 'P=(A+O)', 'Q=(A+D+O)', 'R=(A+E+O)', 'S=(A+F+O)']

                                df = pd.DataFrame(data, columns)
                                
                                os.remove(temp_path)

                                # Converter colunas de remuneração para valor numérico float
                                value_columns = [
                                    'A',
                                    'K=(A+C)',
                                    'L=(A+C+D)',
                                    'M=(A+C+E)',
                                    'N=(A+C+F)',
                                    'P=(A+O)',
                                    'Q=(A+D+O)',
                                    'R=(A+E+O)', 
                                    'S=(A+F+O)',
                                ]
                                for col in value_columns:  # Indicar quais colunas examinar
                                    if col in df.columns:
                                        df[col] = pd.to_numeric(df[col], errors='coerce')

                                print(df.columns)

                                df = df.rename(columns=column_mapping)

                                # Extrai apenas as colunas desejadas
                                df_extracted = df[columns_to_extract + ['Data']]
                                df_extracted['Data'] = pos

                                all_data.append(df_extracted)

                            except Exception as e:
                                print(f"Erro ao ler: {name}\n{e}")

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df, df
    else:
        return None

In [4]:
# Download arquivos de dados zipados
# url = 'https://www.gov.br/servidor/pt-br/observatorio-de-pessoal-govbr/tabela-de-remuneracao-dos-servidores-publicos-federais-civis-e-dos-ex-territorios'
# save_folder = 'C://TabRemun'

# download_ods_files(url, save_folder)

In [ ]:
df

In [5]:
folder_path = 'C://TabRemun'
search_string = 'FIOCRUZ'

columns_to_extract = ['PADRAO', 'NIVEL', 'A', 'B', 'VZ1' , 'C','D' ,'E', 'F','G=(A+B)', 'VZ2', 'H=(A+B+D)', 'I=(A+B+E)', 'J=(A+B+F)', 'VZ5', 'K=(A+C)', 'L=(A+C+D)', 'M=(A+C+E)', 'VZ3', 'N=(A+C+F)', 'VZ4', 'O', 'P=(A+O)', 'Q=(A+D+O)', 'R=(A+E+O)', 'S=(A+F+O)']
print(len(columns_to_extract))

df_final, df = extract_data_from_zip(folder_path, search_string, columns_to_extract)

if df_final is not None:
    print(df_final)
else:
    print("Nenhum dado encontrado.")

26

FUNDAÇÃO OSWALDO CRUZ - FIOCRUZ | janeiro/2015 | Nível Superior
Classe anter: SÊNIOR
['SÊNIOR', 'ÚNICO', 8022.79, 2020.0, 2525.0, None, 6366.21, None, 16409.0, None, 16914.0, None, 1262.5, None, 15651.5, None, None, None, None, None, None, None, None, None, None, None]
['SÊNIOR', 'ÚNICO', 8022.79, 2020.0, 2525.0, None, 6366.21, None, 16409.0, None, 16914.0, None, 1262.5, None, 15651.5, None, None, None, None, None, None, None, None, None, None, None]
RangeIndex(start=0, stop=26, step=1)
Erro ao ler: Tabela de Remuneraç╞o 66 Mar 2015 em Ods/25. FIOCRUZ Cargos/Cargo Especialista FIOCRUZ/Especialista  Fiocruz NS.ods
"None of [Index(['PADRAO', 'NIVEL', 'A', 'B', 'VZ1', 'C', 'D', 'E', 'F', 'G=(A+B)',\n       'VZ2', 'H=(A+B+D)', 'I=(A+B+E)', 'J=(A+B+F)', 'VZ5', 'K=(A+C)',\n       'L=(A+C+D)', 'M=(A+C+E)', 'VZ3', 'N=(A+C+F)', 'VZ4', 'O', 'P=(A+O)',\n       'Q=(A+D+O)', 'R=(A+E+O)', 'S=(A+F+O)', 'Data'],\n      dtype='object')] are in the [columns]"

FUNDAÇÃO OSWALDO CRUZ - FIOCRUZ | janei

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
df_final

In [ ]:
# Exemplo de uso
folder_path = 'F://TabRemun'
search_string = 'FIOCRUZ'
columns_to_extract = ["CLASSE", "PADRÃO", "A","B","C","D","E","F","G=(A+B)","H=(A+B+D)","I=(A+B+E)","J=(A+B+F)","K=(A+C)", "L=(A+C+D)", "M=(A+C+E)", "N=(A+C+F)",	"O", "P=(A+O)",	"Q=(A+D+O)", "R=(A+E+O)", "S=(A+F+O)"]
rotulos = {"VB": "A",
           "Teto Atividade Sem RT": "K=(A+C)",
           "Teto Atividade Aperf./Espec.":	"L=(A+C+D)",
           "Teto Atividade Mestre": "M=(A+C+E)",
           "Teto Atividade Doutor": "N=(A+C+F)",
           "Teto Aposentado Sem RT": "P=(A+O)",
           "Teto Aposentado Aperf./Espec.":	"Q=(A+D+O)",
           "Teto Aposentado Mestre": "R=(A+E+O)",
           "Teto Aposentado Doutor": "S=(A+F+O)",
           }

df_final = extract_data_from_zip(folder_path, search_string, columns_to_extract)

if df_final is not None:
    print(df_final)
else:
    print("Nenhum dado encontrado.")

In [ ]:
url_ipca_completo = "https://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/197912|198001|198002|198003|198004|198005|198006|198007|198008|198009|198010|198011|198012|198101|198102|198103|198104|198105|198106|198107|198108|198109|198110|198111|198112|198201|198202|198203|198204|198205|198206|198207|198208|198209|198210|198211|198212|198301|198302|198303|198304|198305|198306|198307|198308|198309|198310|198311|198312|198401|198402|198403|198404|198405|198406|198407|198408|198409|198410|198411|198412|198501|198502|198503|198504|198505|198506|198507|198508|198509|198510|198511|198512|198601|198602|198603|198604|198605|198606|198607|198608|198609|198610|198611|198612|198701|198702|198703|198704|198705|198706|198707|198708|198709|198710|198711|198712|198801|198802|198803|198804|198805|198806|198807|198808|198809|198810|198811|198812|198901|198902|198903|198904|198905|198906|198907|198908|198909|198910|198911|198912|199001|199002|199003|199004|199005|199006|199007|199008|199009|199010|199011|199012|199101|199102|199103|199104|199105|199106|199107|199108|199109|199110|199111|199112|199201|199202|199203|199204|199205|199206|199207|199208|199209|199210|199211|199212|199301|199302|199303|199304|199305|199306|199307|199308|199309|199310|199311|199312|199401|199402|199403|199404|199405|199406|199407|199408|199409|199410|199411|199412|199501|199502|199503|199504|199505|199506|199507|199508|199509|199510|199511|199512|199601|199602|199603|199604|199605|199606|199607|199608|199609|199610|199611|199612|199701|199702|199703|199704|199705|199706|199707|199708|199709|199710|199711|199712|199801|199802|199803|199804|199805|199806|199807|199808|199809|199810|199811|199812|199901|199902|199903|199904|199905|199906|199907|199908|199909|199910|199911|199912|200001|200002|200003|200004|200005|200006|200007|200008|200009|200010|200011|200012|200101|200102|200103|200104|200105|200106|200107|200108|200109|200110|200111|200112|200201|200202|200203|200204|200205|200206|200207|200208|200209|200210|200211|200212|200301|200302|200303|200304|200305|200306|200307|200308|200309|200310|200311|200312|200401|200402|200403|200404|200405|200406|200407|200408|200409|200410|200411|200412|200501|200502|200503|200504|200505|200506|200507|200508|200509|200510|200511|200512|200601|200602|200603|200604|200605|200606|200607|200608|200609|200610|200611|200612|200701|200702|200703|200704|200705|200706|200707|200708|200709|200710|200711|200712|200801|200802|200803|200804|200805|200806|200807|200808|200809|200810|200811|200812|200901|200902|200903|200904|200905|200906|200907|200908|200909|200910|200911|200912|201001|201002|201003|201004|201005|201006|201007|201008|201009|201010|201011|201012|201101|201102|201103|201104|201105|201106|201107|201108|201109|201110|201111|201112|201201|201202|201203|201204|201205|201206|201207|201208|201209|201210|201211|201212|201301|201302|201303|201304|201305|201306|201307|201308|201309|201310|201311|201312|201401|201402|201403|201404|201405|201406|201407|201408|201409|201410|201411|201412|201501|201502|201503|201504|201505|201506|201507|201508|201509|201510|201511|201512|201601|201602|201603|201604|201605|201606|201607|201608|201609|201610|201611|201612|201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/2266?localidades=N1[all]"
url_ipca = "https://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/199212|199301|199302|199303|199304|199305|199306|199307|199308|199309|199310|199311|199312|199401|199402|199403|199404|199405|199406|199407|199408|199409|199410|199411|199412|199501|199502|199503|199504|199505|199506|199507|199508|199509|199510|199511|199512|199601|199602|199603|199604|199605|199606|199607|199608|199609|199610|199611|199612|199701|199702|199703|199704|199705|199706|199707|199708|199709|199710|199711|199712|199801|199802|199803|199804|199805|199806|199807|199808|199809|199810|199811|199812|199901|199902|199903|199904|199905|199906|199907|199908|199909|199910|199911|199912|200001|200002|200003|200004|200005|200006|200007|200008|200009|200010|200011|200012|200101|200102|200103|200104|200105|200106|200107|200108|200109|200110|200111|200112|200201|200202|200203|200204|200205|200206|200207|200208|200209|200210|200211|200212|200301|200302|200303|200304|200305|200306|200307|200308|200309|200310|200311|200312|200401|200402|200403|200404|200405|200406|200407|200408|200409|200410|200411|200412|200501|200502|200503|200504|200505|200506|200507|200508|200509|200510|200511|200512|200601|200602|200603|200604|200605|200606|200607|200608|200609|200610|200611|200612|200701|200702|200703|200704|200705|200706|200707|200708|200709|200710|200711|200712|200801|200802|200803|200804|200805|200806|200807|200808|200809|200810|200811|200812|200901|200902|200903|200904|200905|200906|200907|200908|200909|200910|200911|200912|201001|201002|201003|201004|201005|201006|201007|201008|201009|201010|201011|201012|201101|201102|201103|201104|201105|201106|201107|201108|201109|201110|201111|201112|201201|201202|201203|201204|201205|201206|201207|201208|201209|201210|201211|201212|201301|201302|201303|201304|201305|201306|201307|201308|201309|201310|201311|201312|201401|201402|201403|201404|201405|201406|201407|201408|201409|201410|201411|201412|201501|201502|201503|201504|201505|201506|201507|201508|201509|201510|201511|201512|201601|201602|201603|201604|201605|201606|201607|201608|201609|201610|201611|201612|201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/2266?localidades=N1[all]"
# url_tab_var = "https://apisidra.ibge.gov.br/values/t/1737/n1/all/v/2266/p/all"
indices_mensais = {}

response = requests.get(url_ipca_completo)
if response.status_code == 200:
    print('Requisição realizada com sucesso!')
response.raise_for_status()

dados_ipca = json.loads(response.content)
indices_ipca = dados_ipca[0]['resultados'][0].get('series')[0].get('serie')
print(f'{len(indices_ipca)} índices IPCA mensais extraídos do IBGE')

# Cria uma lista de tuplas a partir dos itens do dicionário
lista_dados = list(indices_ipca.items())

# Cria o DataFrame
df = pd.DataFrame(lista_dados, columns=["AnoMes", "Valor"])

In [ ]:
df

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Dados fornecidos
anos = list(range(2010, 2027))
inflacao = [0.02, 0.015, 0.025, 0.029, 0.018, 0.02, 0.015, 0.025, 0.029, 0.018, 0.012, 0.047, 0.07, 0.05, 0.03, 0.025, 0.02]
ajuste = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0.2, 0.2]

# Calcula os valores reais ajustados
valores_reais = [100]  # Começa com 100 em 2015
for i in range(1, len(anos)):
    valor_anterior = valores_reais[i - 1]
    valor_atual = valor_anterior * (1 - inflacao[i - 1]) * (1 + ajuste[i - 1])
    valores_reais.append(valor_atual)

# Cria um DataFrame
df = pd.DataFrame({'Anos': anos, 'Valores em Reais': valores_reais})

# Cria o gráfico de barras com Plotly
fig = go.Figure(data=[go.Bar(x=df['Anos'], y=df['Valores em Reais'])])

# Personaliza o layout
fig.update_layout(
    title='Poder de Compra Real Ajustado (2010-2026)',
    xaxis_title='Ano',
    yaxis_title='Valor Real (R$)',
    xaxis=dict(
        dtick=1,  # Garante que cada ano seja mostrado
        tickmode='linear'  # Modo linear para ticks
    )
)

# Atualiza o gráfico para adicionar rótulos de dados
fig.update_traces(texttemplate='%{y:.2f}', textposition='inside')

# Exibe o gráfico
fig.show()


In [ ]:
# dados_ipca

In [ ]:
def plotar_perdas(ano_inicio, ano_final, reajustes_anuais):
    url_ipca_completo = "https://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/197912|198001|198002|198003|198004|198005|198006|198007|198008|198009|198010|198011|198012|198101|198102|198103|198104|198105|198106|198107|198108|198109|198110|198111|198112|198201|198202|198203|198204|198205|198206|198207|198208|198209|198210|198211|198212|198301|198302|198303|198304|198305|198306|198307|198308|198309|198310|198311|198312|198401|198402|198403|198404|198405|198406|198407|198408|198409|198410|198411|198412|198501|198502|198503|198504|198505|198506|198507|198508|198509|198510|198511|198512|198601|198602|198603|198604|198605|198606|198607|198608|198609|198610|198611|198612|198701|198702|198703|198704|198705|198706|198707|198708|198709|198710|198711|198712|198801|198802|198803|198804|198805|198806|198807|198808|198809|198810|198811|198812|198901|198902|198903|198904|198905|198906|198907|198908|198909|198910|198911|198912|199001|199002|199003|199004|199005|199006|199007|199008|199009|199010|199011|199012|199101|199102|199103|199104|199105|199106|199107|199108|199109|199110|199111|199112|199201|199202|199203|199204|199205|199206|199207|199208|199209|199210|199211|199212|199301|199302|199303|199304|199305|199306|199307|199308|199309|199310|199311|199312|199401|199402|199403|199404|199405|199406|199407|199408|199409|199410|199411|199412|199501|199502|199503|199504|199505|199506|199507|199508|199509|199510|199511|199512|199601|199602|199603|199604|199605|199606|199607|199608|199609|199610|199611|199612|199701|199702|199703|199704|199705|199706|199707|199708|199709|199710|199711|199712|199801|199802|199803|199804|199805|199806|199807|199808|199809|199810|199811|199812|199901|199902|199903|199904|199905|199906|199907|199908|199909|199910|199911|199912|200001|200002|200003|200004|200005|200006|200007|200008|200009|200010|200011|200012|200101|200102|200103|200104|200105|200106|200107|200108|200109|200110|200111|200112|200201|200202|200203|200204|200205|200206|200207|200208|200209|200210|200211|200212|200301|200302|200303|200304|200305|200306|200307|200308|200309|200310|200311|200312|200401|200402|200403|200404|200405|200406|200407|200408|200409|200410|200411|200412|200501|200502|200503|200504|200505|200506|200507|200508|200509|200510|200511|200512|200601|200602|200603|200604|200605|200606|200607|200608|200609|200610|200611|200612|200701|200702|200703|200704|200705|200706|200707|200708|200709|200710|200711|200712|200801|200802|200803|200804|200805|200806|200807|200808|200809|200810|200811|200812|200901|200902|200903|200904|200905|200906|200907|200908|200909|200910|200911|200912|201001|201002|201003|201004|201005|201006|201007|201008|201009|201010|201011|201012|201101|201102|201103|201104|201105|201106|201107|201108|201109|201110|201111|201112|201201|201202|201203|201204|201205|201206|201207|201208|201209|201210|201211|201212|201301|201302|201303|201304|201305|201306|201307|201308|201309|201310|201311|201312|201401|201402|201403|201404|201405|201406|201407|201408|201409|201410|201411|201412|201501|201502|201503|201504|201505|201506|201507|201508|201509|201510|201511|201512|201601|201602|201603|201604|201605|201606|201607|201608|201609|201610|201611|201612|201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/2266?localidades=N1[all]"
    url_ipca = "https://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/199212|199301|199302|199303|199304|199305|199306|199307|199308|199309|199310|199311|199312|199401|199402|199403|199404|199405|199406|199407|199408|199409|199410|199411|199412|199501|199502|199503|199504|199505|199506|199507|199508|199509|199510|199511|199512|199601|199602|199603|199604|199605|199606|199607|199608|199609|199610|199611|199612|199701|199702|199703|199704|199705|199706|199707|199708|199709|199710|199711|199712|199801|199802|199803|199804|199805|199806|199807|199808|199809|199810|199811|199812|199901|199902|199903|199904|199905|199906|199907|199908|199909|199910|199911|199912|200001|200002|200003|200004|200005|200006|200007|200008|200009|200010|200011|200012|200101|200102|200103|200104|200105|200106|200107|200108|200109|200110|200111|200112|200201|200202|200203|200204|200205|200206|200207|200208|200209|200210|200211|200212|200301|200302|200303|200304|200305|200306|200307|200308|200309|200310|200311|200312|200401|200402|200403|200404|200405|200406|200407|200408|200409|200410|200411|200412|200501|200502|200503|200504|200505|200506|200507|200508|200509|200510|200511|200512|200601|200602|200603|200604|200605|200606|200607|200608|200609|200610|200611|200612|200701|200702|200703|200704|200705|200706|200707|200708|200709|200710|200711|200712|200801|200802|200803|200804|200805|200806|200807|200808|200809|200810|200811|200812|200901|200902|200903|200904|200905|200906|200907|200908|200909|200910|200911|200912|201001|201002|201003|201004|201005|201006|201007|201008|201009|201010|201011|201012|201101|201102|201103|201104|201105|201106|201107|201108|201109|201110|201111|201112|201201|201202|201203|201204|201205|201206|201207|201208|201209|201210|201211|201212|201301|201302|201303|201304|201305|201306|201307|201308|201309|201310|201311|201312|201401|201402|201403|201404|201405|201406|201407|201408|201409|201410|201411|201412|201501|201502|201503|201504|201505|201506|201507|201508|201509|201510|201511|201512|201601|201602|201603|201604|201605|201606|201607|201608|201609|201610|201611|201612|201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/2266?localidades=N1[all]"
    # url_tab_var = "https://apisidra.ibge.gov.br/values/t/1737/n1/all/v/2266/p/all"
    indices_mensais = {}

    response = requests.get(url_ipca_completo)
    if response.status_code == 200:
        print('Requisição realizada com sucesso!')
    response.raise_for_status()

    dados_ipca = json.loads(response.content)
    indices_ipca = dados_ipca[0]['resultados'][0].get('series')[0].get('serie')
    print(f'{len(indices_ipca)} índices IPCA mensais extraídos do IBGE')

    # Cria uma lista de tuplas a partir dos itens do dicionário
    lista_dados = list(indices_ipca.items())

    # Cria o DataFrame
    df = pd.DataFrame(lista_dados, columns=["AnoMes", "Valor"])    

    # 1. Converter a coluna `Valor` para o tipo numérico
    df['Valor'] = pd.to_numeric(df['Valor'])

    # # Filtrar o DataFrame para o período desejado
    # df = df[
    #     (int(str(df['AnoMes'])[:4]) >= ano_inicio) & (int(str(df['AnoMes'])[:4]) <= ano_final)
    # ]

    # 2. Criar uma nova coluna `Ano` extraindo o ano da coluna `AnoMes`
    df['Ano'] = df['AnoMes'].astype(str).str[:4].astype(int)

    # 3. Calcular a variação percentual mensal do IPCA
    df['InflacaoMensal'] = df['Valor'].pct_change() * -100
    df['InflacaoMensal'] = df['InflacaoMensal'].fillna(0)

    # 4. Calcular a inflação acumulada por ano
    df_inflacao_anual = df.groupby('Ano')['InflacaoMensal'].sum().reset_index()
    df_inflacao_anual = df_inflacao_anual.rename(columns={'InflacaoMensal': 'PerdaInflacao'})

    # 5. Criar um dicionário com anos e correções
    anos = df_inflacao_anual['Ano'].unique().tolist()
    correcoes_salariais_lista = [0 for x in anos]  # Lista de zeros por padrão
    if reajustes_anuais:  # Verificar se a lista de reajustes não está vazia
        correcoes_salariais_lista = reajustes_anuais[:len(anos)]  # Usar os reajustes fornecidos
    correcoes_salariais = dict(zip(anos, correcoes_salariais_lista))

    # 6. Criar um DataFrame a partir das correções salariais
    df_salarios = pd.DataFrame(list(correcoes_salariais.items()), columns=['Ano', 'CorrecaoSalarial'])

    # 7. Fazer o merge entre os DataFrames
    df_merged = df_inflacao_anual.merge(df_salarios, on='Ano', how='left')

    # 8. Preencher os valores NaN da coluna 'CorrecaoSalarial' com 0
    df_merged['CorrecaoSalarial'] = df_merged['CorrecaoSalarial'].fillna(0)

    # 9. Calcular a diferença acumulada entre a correção salarial e a perda por inflação
    df_merged['SaldoAcumulado'] = (df_merged['CorrecaoSalarial'] - df_merged['PerdaInflacao']).cumsum()

    # Filtrar o DataFrame para o período desejado
    df_merged_filtrado = df_merged[
        (df_merged['Ano'] >= ano_inicio) & (df_merged['Ano'] <= ano_final)
    ]

    # Zerar o saldo acumulado no ano de início
    df_merged_filtrado.loc[df_merged_filtrado['Ano'] == ano_inicio, 'SaldoAcumuladoAjustado'] = 0

    # 10. Calcular o valor absoluto máximo para definir o domínio do eixo y principal e secundário
    max_abs_valor = max(
        df_merged_filtrado['PerdaInflacao'].abs().max(),
        df_merged_filtrado['CorrecaoSalarial'].abs().max(),
        df_merged_filtrado['SaldoAcumulado'].abs().max()
    )

    # Preparar os dados para o gráfico de barras
    df_barras = df_merged_filtrado.melt(id_vars='Ano', value_vars=['PerdaInflacao', 'CorrecaoSalarial'], var_name='Tipo', value_name='Valor')

    # Calcular o valor absoluto máximo para definir o domínio do eixo y principal
    max_abs_valor_barras = max(df_merged_filtrado['PerdaInflacao'].abs().max(), df_merged_filtrado['CorrecaoSalarial'].abs().max())

    # Calcular o valor mínimo e máximo do SaldoAcumulado para definir o domínio do eixo y da linha
    min_saldo_acumulado = df_merged_filtrado['SaldoAcumulado'].min()
    max_saldo_acumulado = df_merged_filtrado['SaldoAcumulado'].max()

    # Criar o gráfico de barras
    barras = alt.Chart(df_barras).mark_bar().encode(
        x=alt.X('Ano:O', axis=alt.Axis(labelAngle=-45)),
        y=alt.Y('Valor:Q', axis=alt.Axis(title='Variação (%)', titleColor='black'), scale=alt.Scale(domain=[-max_abs_valor_barras, max_abs_valor_barras])),
        color=alt.Color('Tipo:N', scale={'domain': ['PerdaInflacao', 'CorrecaoSalarial'], 'range': ['red', 'steelblue']}),
        tooltip=['Ano', 'Valor', 'Tipo']
    ).properties(
        title='Perda por Inflação, Reposição Salarial e Saldo Acumulado'
    )

    # Adicionar os rótulos às barras
    text = barras.mark_text(
        align='center',
        baseline='bottom',
        dy=-5
    ).encode(
        text=alt.Text('Valor:Q', format='.2f')
    )

    # Criar o gráfico de linha para o saldo acumulado (eixo secundário)
    linha = alt.Chart(df_merged_filtrado).mark_line(color='green', point=True).encode(
        x='Ano:O',
        y=alt.Y('SaldoAcumulado:Q', axis=alt.Axis(title='Saldo Acumulado (%)', titleColor='green'), scale=alt.Scale(domain=[min_saldo_acumulado, max_saldo_acumulado])),
        tooltip=['Ano', 'SaldoAcumulado']
    )

    # Adicionar rótulos de dados à linha
    text_linha = linha.mark_text(
        align='left',
        baseline='middle',
        dx=5,  # Nudge text to right so it doesn't overlap with the line
        color='black'
    ).encode(
        text=alt.Text('SaldoAcumulado:Q', format='.2f')
    )

    # Combinar os gráficos com os rótulos
    grafico_final = alt.layer(barras + text, linha + text_linha).resolve_scale(
        y='independent'
    ).interactive()


    grafico_final = grafico_final.properties(
        height=600,  # Largura de 800 pixels
        width=1100  # Largura de 1200 pixels
    )

    return df_merged, grafico_final

# Exemplo de uso
ano_inicio = 2003
ano_final = 2023
reajustes_anuais = [2.5, 3.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5]  # Exemplo de reajustes

df_merged, grafico_final = plotar_perdas(ano_inicio, ano_final, reajustes_anuais)
grafico_final

In [ ]:
import altair as alt
alt.renderers.enable('jupyterlab')
grafico_final

In [ ]:
# Converter a coluna `Valor` para o tipo numérico
df['Valor'] = pd.to_numeric(df['Valor'])

# Extrair o ano da coluna `AnoMes`
df['Ano'] = df['AnoMes'].astype(str).str[:4].astype(int)

# Filtrar para manter apenas os meses de dezembro
df_dezembro = df[df['AnoMes'].astype(str).str[-2:] == '12'].copy()

# Agrupar por ano e selecionar o primeiro valor de cada ano
df_anual = df_dezembro.groupby('Ano')['Valor'].first().reset_index()

# Obter o ano de início e o ano de término do usuário
ano_inicio = int(input("Digite o ano de início: "))
ano_termino = int(input("Digite o ano de término: "))

# Filtrar para o período desejado
df_periodo = df_anual[(df_anual['Ano'] >= ano_inicio) & (df_anual['Ano'] <= ano_termino)]

# Calcular a variação acumulada do IPCA
variacao_acumulada = (df_periodo['Valor'].iloc[-1] / df_periodo['Valor'].iloc[0] - 1) * 100

# Calcular a taxa anual equivalente
num_anos = ano_termino - ano_inicio + 1
taxa_anual = ((1 + variacao_acumulada / 100) ** (1 / num_anos) - 1) * 100

# Imprimir os resultados
print(f'Variação acumulada do IPCA de {ano_inicio} a {ano_termino}: {variacao_acumulada:.2f}%')
print(f'Taxa anual equivalente: {taxa_anual:.2f}%')

# Filtrar para o período desejado
df_periodo = df_anual[(df_anual['Ano'] >= ano_inicio) & (df_anual['Ano'] <= ano_termino)]

# Calcular a variação acumulada do IPCA
variacao_acumulada = (df_periodo['Valor'].iloc[-1] / df_periodo['Valor'].iloc[0] - 1) * 100

# Calcular a taxa anual equivalente
num_anos = ano_termino - ano_inicio + 1
taxa_anual = ((1 + variacao_acumulada / 100) ** (1 / num_anos) - 1) * 100

# Imprimir os resultados
print(f'Variação acumulada do IPCA de {ano_inicio} a {ano_termino}: {variacao_acumulada:.2f}%')
print(f'Taxa anual equivalente: {taxa_anual:.2f}%')

In [ ]:
# Converter a coluna `Valor` para o tipo numérico
df['Valor'] = pd.to_numeric(df['Valor'])

# Extrair o ano da coluna `AnoMes`
df['Ano'] = df['AnoMes'].astype(str).str[:4].astype(int)

# Filtrar para manter apenas os meses de dezembro
df_dezembro = df[df['AnoMes'].astype(str).str[-2:] == '12'].copy()

# Agrupar por ano e selecionar o primeiro valor de cada ano
df_anual = df_dezembro.groupby('Ano')['Valor'].first().reset_index()

# Obter o ano de início e o ano de término do usuário
ano_inicio = int(input("Digite o ano de início: "))
ano_termino = int(input("Digite o ano de término: "))

# Filtrar para o período desejado
df_periodo = df_anual[(df_anual['Ano'] >= ano_inicio) & (df_anual['Ano'] <= ano_termino)]

# Calcular a variação acumulada do IPCA
variacao_acumulada = (df_periodo['Valor'].iloc[-1] / df_periodo['Valor'].iloc[0] - 1) * 100

# Calcular a taxa anual equivalente
num_anos = ano_termino - ano_inicio + 1
taxa_anual = ((1 + variacao_acumulada / 100) ** (1 / num_anos) - 1) * 100

# Imprimir os resultados
print(f'Variação acumulada do IPCA de {ano_inicio} a {ano_termino}: {variacao_acumulada:.2f}%')
print(f'Taxa anual equivalente: {taxa_anual:.2f}%')

In [ ]:
import altair as alt

# Criar um DataFrame com os dados de reposição salarial
df_reposicao = df_merged[['Ano', 'CorrecaoSalarial']].rename(columns={'CorrecaoSalarial': 'Valor'})
df_reposicao['Tipo'] = 'Reposição Salarial'

# Criar um DataFrame com os dados de perda por inflação
df_perda_inflacao = df_merged[['Ano', 'InflacaoPercentual']].rename(columns={'InflacaoPercentual': 'Valor'})
df_perda_inflacao['Valor'] = -df_perda_inflacao['Valor']  # Inverter os valores para ficarem negativos
df_perda_inflacao['Tipo'] = 'Perda por Inflação'

# Concatenar os DataFrames de reposição e perda
df_plot = pd.concat([df_reposicao, df_perda_inflacao])

# Calcular o valor absoluto máximo para definir o domínio do eixo y principal
max_abs_valor = df_plot['Valor'].abs().max()

# Criar o gráfico de barras
barras = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X('Ano:O', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Valor:Q', scale=alt.Scale(domain=[-max_abs_valor, max_abs_valor])),  # Ajustar escala y
    color=alt.Color('Tipo:N', scale={'domain': ['Perda por Inflação', 'Reposição Salarial'], 'range': ['red', 'steelblue']}),
    tooltip=['Ano', 'Valor', 'Tipo']
).properties(
    title='Reposição Salarial, Perda por Inflação e Saldo Acumulado'
)

# Adicionar os rótulos às barras
text = barras.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Nudge text upwards so it doesn't overlap with the bar
).encode(
    text=alt.Text('Valor:Q', format='.2f')
)

# Criar o gráfico de linha para o saldo acumulado (eixo secundário)
linha = alt.Chart(df_merged).mark_line(color='green', point=True).encode(
    x='Ano:O',
    y=alt.Y('DiferencaAcumulada:Q', axis=alt.Axis(title='Saldo Acumulado', titleColor='green')),
    tooltip=['Ano', 'DiferencaAcumulada']
)

# Combinar os gráficos
grafico_final = alt.layer(barras + text, linha).resolve_scale(
    y = 'independent'
).interactive()

# Exibir o gráfico
grafico_final

grafico_final = grafico_final.properties(
    height=600,  # Largura de 800 pixels
    width=1100  # Largura de 1200 pixels
)
# Salve o gráfico em um arquivo JSON
grafico_final.save('grafico_perda_ganho_poder_compra.json')

In [ ]:
import altair as alt
alt.renderers.enable('jupyterlab')
grafico_final

In [ ]:
df_merged

In [ ]:
from matplotlib import pyplot as plt

plt.plot(indices_ipca.values())

In [ ]:
len(dados_ipca[0])

In [ ]:
import requests
import pandas as pd
import json
import csv
from io import BytesIO, TextIOWrapper

url_tab_var = "https://apisidra.ibge.gov.br/values/t/1737/n1/all/v/2266/p/all"
indices_mensais = {}

response = requests.get(url_tab_var)
response.raise_for_status()

meses=[]
indices=[]

# Ler o CSV com TextIOWrapper para converter bytes em strings
with TextIOWrapper(BytesIO(response.content), encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=";")

    # Extrair dados relevantes e construir dicionário
    for row in reader:
        print(row)
        if "D3C" in row:
            meses.append(row)
        if "V" in row:
            indices.append(row)

In [ ]:
len(meses)

In [ ]:
len(indices)

In [ ]:
# !pip install sidrapy

In [ ]:
import sidrapy

# Importa os dados do SIDRA
ipca_raw = sidrapy.get_table(table_code= "1737",
                            territorial_level = "1",
                            ibge_territorial_code = "all",
                            period = "all",
                            )

# Checamos a importação
ipca_raw.head()

In [ ]:
import pandas as pd

# Converter a coluna D3C para numérico, ignorando valores não numéricos
ipca_raw['D3C'] = pd.to_numeric(ipca_raw['D3C'], errors='coerce')

# Filtrar as linhas onde D3C é igual a 2265
filtro = ipca_raw['D3C'] == 2265
ipca_filtrado = ipca_raw[filtro]

ipca_filtrado[["V","D2C"]]

In [ ]:
import sidrapy

# Importa os dados do SIDRA
pib_sa_raw = sidrapy.get_table(table_code= "1621",
                            territorial_level = "1",
                            ibge_territorial_code = "all",
                            period = "all",
                            classification = "11255/90707")

# Checamos a importação
pib_sa_raw.head()

In [ ]:
from poder_compra import IPCAData

ipca = IPCAData()
meses, indices = ipca.obter_numeros_indice()
for i, j in zip(meses, indices):
    print(i,j)


In [ ]:
indices[:3]

In [ ]:
meses[0]

In [ ]:
import json
a,b = indices[3].split(": ")
{a.replace("\"",""): (b.replace("\"","").replace(",",""))}

In [ ]:
a.replace("\"","")

In [ ]:
float(b.replace("\"","").replace(",",""))

In [ ]:
# Extrair dados relevantes e construir dicionário
for row in reader:
    if row[2] == "MN" and row[1] == "Número-índice":
        data = row[4270].replace(" ", "-") 
        valor = float(row[3].replace(",", "."))
        indices_mensais[data] = valor

In [ ]:
indices_mensais

In [ ]:
from poder_compra import IPCAData

ipca_data = IPCAData()
poder_compra = PoderCompra(ipca_data.dados_ipca)
perdas = poder_compra.calcular_perdas()

previsao = PrevisaoInflacao(ipca_data.dados_ipca)
inflacao_futura = previsao.prever()

Visualizacao.plotar_evolucao_poder_compra(ipca_data.dados_ipca, perdas)